In [7]:
# ──────────────────────────────────────────────────────────────
# Big‑Idea “Round‑Out” notebook builder  •  v2 (2025‑07‑21)
# ──────────────────────────────────────────────────────────────
import nbformat as nbf, zipfile, csv, os, math, textwrap
from datetime import datetime

# ── Helper: make a new notebook cell list ─────────────────────
nb = nbf.v4.new_notebook(); cells = []

def code(src:str):      cells.append(nbf.v4.new_code_cell(textwrap.dedent(src)))
def markdown(md:str):   cells.append(nbf.v4.new_markdown_cell(textwrap.dedent(md)))

# ── 1 • Title & instructions ─────────────────────────────────
markdown("""
# Big‑Idea Round‑Out Notebook

Self‑contained runner that:

* validates numeric identities (50‑digit precision)
* draws a **fully‑filled or auto‑cropped prime‑sieve heat‑map** mod 137
* writes CSV/PNG outputs and bundles them into a ZIP for journal upload

Just **Run all**. Add or edit identities in *Cell 5*.
""")

# ── 2 • Imports & precision ──────────────────────────────────
code("""
from decimal import Decimal, getcontext
import numpy as np, matplotlib.pyplot as plt, csv, zipfile, os
getcontext().prec = 60  # 60‑digit decimal precision
""")

# ── 3 • Fundamental constants ────────────────────────────────
code("""
pi  = Decimal('3.14159265358979323846264338327950288419716939937510')
phi = Decimal('1.61803398874989484820458683436563811772030917980576')
alpha_inv = Decimal('137.035999084')       # CODATA 2018
sqrt2, sqrt3 = Decimal(2).sqrt(), Decimal(3).sqrt()
print('Constants initialised.'); print()
""")

# ── 4 • Utility functions  (delta, check) ────────────────────
code("""
def delta(a: Decimal, b: Decimal) -> Decimal:
    return abs(a - b)

def check(label, lhs: Decimal, rhs: Decimal, tol=Decimal('1e-50')):
    d = delta(lhs, rhs)
    ok = d <= tol
    badge = '✓' if ok else '✗'
    print(f'{badge}  {label:<20}  Δ = {d}')
    return [label, str(lhs), str(rhs), str(d), 'PASS' if ok else 'FAIL']
""")

# ── 5 • Identity tests (edit here) ───────────────────────────
code("""
# ▼▼  Replace or extend this list with your own identities  ▼▼
identities = [
    ('π² vs 2πφ',          pi**2,           2*pi*phi),
    ('φ² vs π-1',           phi**2,          pi - Decimal(1)),
]

rows = [['label', 'lhs', 'rhs', 'delta', 'status']]
for lab, lhs, rhs in identities:
    rows.append(check(lab, lhs, rhs))

with open('identity_results.csv', 'w', newline='') as f:
    csv.writer(f).writerows(rows)
print('\\nidentity_results.csv saved.'); print()
""")

# ── 6 • Prime‑sieve heat‑map (auto mode / crop mode) ─────────
code("""
import math, itertools
# ---------- settings ----------
limit = 200_000        # sieve upper bound   ≈ 18k primes
grid_mode = 'autosize' # 'autosize' | 'crop' | 'fixed'
fixed_size = 100       # used only if grid_mode == 'fixed'
# ------------------------------

# prime sieve
sieve = np.ones(limit, dtype=bool)
primes = []
for p in range(2, limit):
    if sieve[p]:
        primes.append(p)
        sieve[p*p::p] = False

# decide grid size
if grid_mode == 'autosize':
    size = math.ceil(math.sqrt(len(primes)))
elif grid_mode == 'fixed':
    size = fixed_size
else:  # 'crop'
    size = fixed_size

mat = np.zeros((size, size))
for i, p in enumerate(primes[: size*size ]):
    mat[i // size, i % size] = p % 137

# crop empty rows/cols if requested
if grid_mode == 'crop':
    nz_rows = np.where(mat.sum(axis=1) != 0)[0]
    nz_cols = np.where(mat.sum(axis=0) != 0)[0]
    mat = mat[nz_rows[:, None], nz_cols]

plt.figure(figsize=(6,6))
plt.imshow(mat, interpolation='nearest')
plt.axis('off')
plt.title(f'Prime residues mod 137  • {len(primes):,} primes  • {grid_mode}')
plt.savefig('prime_heatmap.png', dpi=300, bbox_inches='tight')
plt.close(); print('prime_heatmap.png saved.'); print()
""")

# ── 7 • Bundle artefacts ─────────────────────────────────────
code("""
with open('README.txt','w') as f:
    f.write('Big‑Idea Reproducibility Bundle\\n')
    f.write('Generated: ' + datetime.utcnow().isoformat() + 'Z\\n\\n')
    f.write('FILES:\\n- identity_results.csv\\n- prime_heatmap.png\\n')

with zipfile.ZipFile('big_idea_bundle.zip', 'w') as z:
    for fname in ['identity_results.csv', 'prime_heatmap.png', 'README.txt']:
        z.write(fname)
print('big_idea_bundle.zip created.  All artefacts ready!')
""")

# ── 8 • Write notebook file & finish ─────────────────────────
nb['cells'] = cells
outfile = '/content/big_idea_roundout.ipynb'
nbf.write(nb, outfile)
print(f'Notebook written → {outfile}')


Notebook written → /content/big_idea_roundout.ipynb
